In [0]:
from pylab import *
from scipy import fft
import matplotlib.pyplot as plt
from scipy.signal import periodogram as psd
import numpy as np
from scipy.signal import find_peaks
from math import sqrt, sin
from cmath import pi


%matplotlib inline

Motorized stage details (https://www.thorlabs.com/newgrouppage9.cfm?objectgroup_id=2163): 


*   Max Velocity = 2.6 mm/s
*   Max Travel Range = 12mm (12e3 µm)
*   Minimum Step Size = 0.10 µm

*   inv step size: 10000 Hz (= $\frac{max velocity}{step size}$)

Review this document for understanding of how FTS works/ data looks: https://chem.libretexts.org/Bookshelves/Physical_and_Theoretical_Chemistry_Textbook_Maps/Supplemental_Modules_(Physical_and_Theoretical_Chemistry)/Spectroscopy/Fundamentals_of_Spectroscopy/The_Power_of_the_Fourier_Transform_for_Spectroscopists

### Want to compare different signals on the order of our expected magnitude: 

*   Source off (noise on the order of 2e-8  W)
*   Source on (blackbody spectrum with noise on the order of 1e-7W)

In [0]:
#This code, as borrowed from the Power Law Relationship file calcualtes the power (W)  of blackbody spectrum for a given frequncy 
#fixed variables
k_B = 8.617333262145e-5 #eV K^{-1}
h = 4.135667696e-15 #eV * s
c = 299792458 #m / s
eV_to_J = 1.602176565e-19 
Temp = 2000 #K

#modifiable variables
r_rad_small = 0.001 #m -radius of radiator
r_abs = 0.001 #m -radius of absorber
d =  0.1 #m -distance between absorber and radiator ?

#a is lower lim, b is upper lim, n is number of slices
def simpson(g, a, b, n): 
    h=(b-a)/n
    k=0.0
    x=a + h
    for i in arange(1,n/2 + 1):
        k += 4*g(x)
        x += 2*h
    x = a + 2*h
    for i in arange(1,n/2):
        k += 2*g(x)
        x += 2*h
    r = (h/3)*(g(a)+g(b)+k)
    return r
def gamma_0(T, r_rad):  
    area = pi * r_abs**2
    sin_theta = r_rad /np.sqrt(r_rad**2 + d**2)
    return area*sin_theta * (8 * pi * (k_B * T)**2)/(h**3 * c**2)

#function to integrate via Sumpson's rule
#Noah's modifications to be overflow safe
def power_integral(x):
    #if we have a list, convert to numpy array
    if(type(x)==list):
        x = np.array(dtype=np.float128)
        
    #if we have a numpy array, proceed
    if(type(x)== np.ndarray):
        retvals = np.zeros_like(x,dtype=np.float128)
        inds = x < 1e3
        #small values can be directly calculated
        retvals[inds] = x[inds]**3/(np.exp(x[inds])-1.0)
        #in the limit of large x, f(x)~exp(-x)
        retvals[~inds] = np.exp(-x[~inds])
        return retvals
    
    #assume we have a number
    else: 
        if(type(x)!='float128'):
            x=np.float128(x)
        if(x>1e3):
            return np.exp(-x)
        else:
            return x**3/(np.exp(x)-1.0)

#this function returns power, given bounds in terms of energy
def power_func(T, a, b, r_rad):
    start = a/(k_B * T)  #lowerbound
    end = b/(k_B * T) #upperbound
    n = 100 #slices
    return eV_to_J * k_B**2 * T**2 * gamma_0(T, r_rad)*simpson(power_integral,start,end,n)
def power_intensity(v, slice_size):
    E_min = (v-slice_size)*h #eV 
    E_max = (v+slice_size)*h #eV 
    #print("e = " , v*h, " & E_min =", E_min," & E_max =", E_max )
    power = power_func(Temp, E_min, E_max, r_rad_small)
    return power

In [0]:
arm_length_1 = 15 #cm
arm_length_2 = 10 #cm
cm_to_micron = 10000
dx = abs(arm_length_1-arm_length_2)*cm_to_micron

adjustment = 1e-7

dt = 0.1 #micron, step size
fs = 1/dt #micron^-1, inv. step size (multiply by h gives energy resolution)
T = 12e3 #micron, total travel length
sigma = 2e-8 #noise offset intensity
peaks = []
weights_nofilter = []
c_micron_per_second = 3e8 * 1e6
Hz_to_THz = 1e-12
peak_thershold = 1e-6 #increase this as noise increases, to ensure we are not accidnetally claiming the noise as a peak

fig = plt.figure(figsize=(18, 12), dpi=150)  

#for blackbody spectrum, decide frequeny range and number of frequency slices 
min_frequency = 0.01 #THz
max_frequency = 3100 #THz
num_slices = 10000 #1e6
frequency_slice_size = (max_frequency - min_frequency)/num_slices #micron
frequencies = arange(min_frequency, max_frequency, frequency_slice_size)


#This section of code creates the signal
for nu in frequencies: #(THz)
  weights_nofilter.append(power_intensity(nu, frequency_slice_size)) # for blackbody, this is the blackbody intensity in W
plt.subplot(2, 2, 1)
plt.plot(frequencies, weights_nofilter , 'o')

x=np.arange(0,T,dt)
y_phaseNoise = np.zeros_like(x)
for nu in range(len(frequencies)):
  k = frequencies[nu]/(Hz_to_THz*c_micron_per_second) #get wavenumber
  phase = 2*pi*dx*k #phase offset depends on wavenumber
  y_phaseNoise += weights_nofilter[nu]*1/2*(np.cos(2*pi*k*x + phase)+ 1)
y = y_phaseNoise + np.random.rand(len(x))*sigma #create the input signal


plt.subplot(2, 2, 2)
plt.plot(x,y,label= T)

plt.subplot(2, 2, 3)
f_nofilter,p_nofilter = psd(y,fs=fs) #wavenumber, spectrum
plt.semilogy(f_nofilter, np.sqrt(p_nofilter), label=T)
peaks_index, _ = find_peaks(np.log10(np.sqrt(p_nofilter)), prominence=(2.3))
plt.plot(f_nofilter[peaks_index], np.sqrt(p_nofilter[peaks_index]), "x")
peaks.append(f_nofilter[peaks_index])

p_adjusted_nofilter = np.sqrt(p_nofilter) - adjustment
for ps in range(len(p_adjusted_nofilter)):
  if p_adjusted_nofilter[ps] < 0:
    p_adjusted_nofilter[ps] = 0

plt.subplot(2, 2, 4)
plt.semilogy(f_nofilter*c_micron_per_second*Hz_to_THz, np.sqrt(p_nofilter),label=T)

#plot details for blackbody spectrum plot
plt.subplot(2, 2, 1)
plt.grid(True)
plt.yscale('log')
title = "Blackbody Spectrum: Weighting vs. Frequncy of Signal \n (T = " + str(Temp) + "K, Frequency spacing = "+ str(round(frequency_slice_size,2)) + " THz)"
plt.title(title)
plt.xlabel("Frequency (THz)")
plt.ylabel("Spectral Radiance (W) \n Weight")

#plot details for signal plot
plt.subplot(2, 2, 2)
plt.grid(True)
plt.title("Signal of Blackbody Spectrum")
plt.legend(title="Travel Length ($\mu m$)")
plt.legend(title="Travel Length ($\mu m$)")
plt.xlabel("Optical path difference ($\mu m$)")
plt.ylabel("Signal Amplitude \n Corresponds to optical power (W)")

#plot details for fourier transform plot
plt.subplot(2, 2, 3)
plt.title("Transform as a function of Wavenumber")
plt.grid(True)
plt.legend(title="Travel Length ($\mu m$)")
plt.xlabel('Wavenumber ($\mu m ^{-1}$)')
plt.ylabel("Fourier Amplitude \n [sqrt(PSD) (Arbitrary Units)]")

#plot details for fourier transform plot in terms of frequency rather than wavenumber
plt.subplot(2, 2, 4)
plt.title("Transform as a function of Frequency")
plt.xscale('log')
plt.grid(True)
plt.legend(title="Travel Length ($\mu m$)")
plt.xlabel('Frequency (THz)')
plt.ylabel("Fourier Amplitude \n [sqrt(PSD) (Arbitrary Units)]")
plt.show()

# print("For total travel distance ", Tlist[0], "µm the peaks occur at frequencies:", peaks[0]*c_micron_per_second*Hz_to_THz , "THz")
# print("There are ", len(peaks[0]), " measured peaks")


<ipython-input-2-5019e1ca8a97>:67: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "power_intensity" failed type inference due to: Invalid use of type(CPUDispatcher(<function power_func at 0x7f1822d60a60>)) with parameters (Literal[int](2000), float64, float64, float64)
 * parameterized
[1] During: resolving callee type: type(CPUDispatcher(<function power_func at 0x7f1822d60a60>))
[2] During: typing of call at <ipython-input-2-5019e1ca8a97> (72)


File "<ipython-input-2-5019e1ca8a97>", line 72:
def power_intensity(v, slice_size):
    <source elided>
    #print("e = " , v*h, " & E_min =", E_min," & E_max =", E_max )
    power = power_func(Temp, E_min, E_max, r_rad_small)
    ^

  @jit
/usr/local/lib/python3.6/dist-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "power_intensity" was compiled in object mode without forceobj=True.

File "<ipython-input-2-5019e1ca8a97>", line 68:
@jit
def power_intensity(v, slice_size):
^

In [0]:
print(max(y))

4.301449349352751e-33
